In [1]:
import gym
import numpy as np
import torch
from torch import FloatTensor
from torch.autograd import Variable
import matplotlib
import matplotlib.pyplot as plt
import time
from collections import deque

from ddqnAgents import QNetAgent
from replay_buffer import ReplayMemory, Transition

# setup matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: 
    from IPython import display
# 动态图
plt.ion()
use_cuda = torch.cuda.is_available()


# 参数设置
device = torch.device("cuda" if use_cuda else "cpu")
BATCH_SIZE = 64
gamma = 0.99
LEARNING_RATE = 0.001
TARGET_UPFATE = 10
num_episodes = 20000
print_every = 10
hidden_dim = 32
min_eps = 0.01
max_eps_episode = 150

# 环境
env = gym.make('CartPole-v1')
# env = gym.wrappers.RecordVideo(env, directory='monitors', force=True)

space_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
print('input_dim: ', space_dim, ', output_dim: ', action_dim, ', hidden_dim: ', hidden_dim)

agent = QNetAgent(n_states=space_dim, n_actions=action_dim, hidden_dim=hidden_dim)


# 权重衰减
def epsilon_annealing(i_episode, max_episode, min_eps:float):
    slope = (min_eps - 1.0) / max_episode
    ret_eps = max(slope*i_episode+1.0, min_eps)
    return ret_eps

# 保存权重
def save(directory, filename):
    torch.save(agent.q_local.state_dict(), '{0}/{1}_local.pth'.format(directory, filename))
    torch.save(agent.q_target.state_dict(), '%s/%s_target.pth' % (directory, filename))


input_dim:  4 , output_dim:  2 , hidden_dim:  32


### 2 Single episode

In [12]:
def run_episode(env, agent, eps):
    """
    params:
        env (gym.Env): gym environment (CartPole-v0)
        agent (Agent): agent will train and get action        
        eps (float): eps-greedy for exploration

    returns:
        int: return earned in this episode
    """
    state = env.reset()[0]
    done = False
    total_reward = 0

    print(state)

    while not done:
        # 根据当前状态采取action
        action = agent.get_action(FloatTensor([state]), eps)

        next_state, reward, done, info, _ = env.step(action)

        total_reward += reward

        # 结束收到惩罚
        if done:
            reward = -1

        # Store the transition in memory
        agent.replay_memory.push(
            (FloatTensor([state]), 
            action,                    # action is already a tensor
            FloatTensor([reward]), 
            FloatTensor([next_state]), 
            FloatTensor([done]))
        )

        

        

        
run_episode(env, agent, eps=0.0001)        

[-0.04846687  0.0403985  -0.03763504  0.02360786]
